In [104]:
import serial 
import numpy as np 
import pandas as pd 
import time
from PIL import Image 

First create the serial connection and open it.  

In [105]:
port = "/dev/cu.usbserial-AL03XDEJ"                              
baud = 115200
  
ser = serial.Serial(port, baud, timeout=1)
ser.bytesize = serial.EIGHTBITS #number of bits per bytes
ser.parity = serial.PARITY_NONE #set parity check: no parity
ser.stopbits = serial.STOPBITS_ONE # set # of stop bits: 1 

 First, read the status of the transciever and print all responses from the transciever.

In [106]:
read_register = 'ES+R2200' 
message = read_register + '\r\n'

# First read the status control register 
ser.write(message.encode('ascii'))
ret = ser.read(size=30)
print('Receiving...', ret.decode('ascii'))


Receiving... OK+0022503303


Before setting the transciever in transmit mode, we want to set the reciever to have a PIPE Timeout setting. This essentially turns off pipe mode if a transmission does not get commanded within that timeframe 

In [107]:
# Second, write to the register 
write_register = "ES+W220600000020"
message = write_register + '\r\n'


ser.write(message.encode('ascii'))
ret = ser.read(size=30)
print('Receiving...', ret.decode('ascii'))

Receiving... OK


__Note: This sets the tranmission timeout time to 32 seconds, meaning that if you don't send a transmit message for 32 seconds, the PIPE mode will timeout.__ Now turn on pipe transmit and transmit 

In [108]:
# Third, write to the register 
write_register = "ES+W22003323"
message = write_register + '\r\n'


ser.write(message.encode('ascii'))
ret = ser.read(size=30)
print('Receiving...', ret.decode('ascii'))

Receiving... OK+3323


Now the transceiver is in transmission mode. __NOTE: Now we are going to transmit. Please make sure that the recieve station is ready for reception and ready to record the signal before transmitting__ 

First we need to import the image and prep it. Prepping it means that we prepare it as ascii data. 

In [109]:
with open("sample-picture.jpg", "rb") as image: 
    temp = image.read()
    data = bytearray(temp)

Now lets split the data into 108 byte packet sizes. 

In [110]:
all_data = [] 
for i in range(0,len(data), 108):
    packet = data[i:i+108]
    all_data.append(packet)

Now we must transmit the image, packet by packet. 

In [112]:
start = time.time()
for i in all_data: 
    message = i
    ser.write(message)
end = time.time()
print(end - start)

5.6819398403167725


Now, before we close the transceiver. We should always restore the orginal Status Control Register state with the ESTTC commands. The original state is always 3303. 

Thus, the command to return to original state is ES+W22003303 

In [84]:
# Second, write to the register 
ret_register = "ES+W22003303"
message = ret_register + '\r\n'

# First read the status control register 
ser.write(message.encode('ascii'))
ret = ser.read(size=30)
print('Receiving...', ret.decode('ascii'))

OK+3303


Finally, we close the serial port connection! Tutorial finished. 

In [41]:
ser.close()